The face encoding model is a ResNet.

To improve recognition accuracy:

1. add jitters to the face encoding, this tells dlib to randomly distort your image, take encodings of all and return the averaged encodings. this will make the algorithm slow though.
2. add unsampling i.e zoomingg to the original image, this will help detect the smaller faces in the image, at the cost of slow run time.


In [3]:
from imutils import face_utils
import dlib
import cv2
import numpy as np
import openface
import face_recognition
import os
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
import keras
from random import shuffle
from keras.layers import Dense, Dropout, Activation, Conv2D, Flatten, MaxPooling2D
from keras.models import Sequential

IMG_SIZE=500

def get_face_embeddings_from_image(image, convert_to_rgb=False):
    """
    Take a raw image and run both the face detection and face embedding model on it
    """
    # Convert from BGR to RGB if needed
    if convert_to_rgb:
        image = image[:, :, ::-1]

    # run the face detection model to find face locations
    face_locations = face_recognition.face_locations(image)
    # face_locations = face_recognition.face_locations(face_image, number_of_times_to_upsample=2)

    # run the embedding model to get face embeddings for the supplied locations
    face_encodings = face_recognition.face_encodings(image, face_locations, num_jitters=10)

    return face_locations, face_encodings

def setup_database():
    """
    Load reference images and create a database of their face encodings
    """
    database = {}
    for filename in glob('training_images_face/*.jpeg'):
        # load image
        image_rgb = face_recognition.load_image_file(filename)

        # use the name in the filename as the identity key
        identity = os.path.splitext(os.path.basename(filename))[0]

        # get the face encoding and link it to the identity
        locations, encodings = get_face_embeddings_from_image(image_rgb)
        database[identity] = encodings[0]

    return database

# identify face through image

def run_face_recognition_image(database):
    known_face_encodings = list(database.values())
    known_face_names = list(database.keys())

    img=cv2.imread('training images/chad-smith/chad3.jpeg')
    face_locations, face_encodings=get_face_embeddings_from_image(img, convert_to_rgb=True)

    for location, face_encoding in zip(face_locations, face_encodings):
        MAX_DISTANCE=0.6
        # get the distances from this encoding to those of all reference images
        distances = face_recognition.face_distance(known_face_encodings, face_encoding)

        # select the closest match (smallest distance) if it's below the threshold value
        if np.any(distances <= MAX_DISTANCE):
            best_match_idx = np.argmin(distances)
            name = known_face_names[best_match_idx]
        else:
            name = None
    print(name)

def run_face_recognition_video(database):
    """
    Start the face recognition via the webcam
    """
    # Open a handler for the camera
    video_capture = cv2.VideoCapture(0)

    # the face_recognitino library uses keys and values of your database separately
    known_face_encodings = list(database.values())
    known_face_names = list(database.keys())
    
    while video_capture.isOpened():
        # Grab a single frame of video (and check if it went ok)
        ok, frame = video_capture.read()
        if not ok:
            logging.error("Could not read frame from camera. Stopping video capture.")
            break
#         if ok:
           
            
        # run detection and embedding models
        face_locations, face_encodings = get_face_embeddings_from_image(frame, convert_to_rgb=True)
        MAX_DISTANCE=0.6
        # Loop through each face in this frame of video and see if there's a match
        for location, face_encoding in zip(face_locations, face_encodings):
            T,R,B,L=location
            cropped_img= frame[T:B, L:R]
            result=real_fake(cropped_img)
            cv2.imwrite("cropped.jpeg",cropped_img) 
            # get the distances from this encoding to those of all reference images
            if result==1:
                distances = face_recognition.face_distance(known_face_encodings, face_encoding)

                # select the closest match (smallest distance) if it's below the threshold value
                if np.any(distances <= MAX_DISTANCE):
                    best_match_idx = np.argmin(distances)
                    name = known_face_names[best_match_idx]
                else:
                    name = None

                # put recognition info on the image
                paint_detected_face_on_image(frame, location, name)
            else:
                paint_detected_face_on_image(frame, location, 'fake')
        # Display the resulting image
        cv2.imshow('Video', frame)

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release handle to the webcam
    video_capture.release()
    cv2.destroyAllWindows()

def paint_detected_face_on_image(frame, location, name=None):
    """
    Paint a rectangle around the face and write the name
    """
    # unpack the coordinates from the location tuple
    top, right, bottom, left = location
    
    if name is 'fake':
        color = (0, 0, 255)  # red for unrecognized face
    elif name is None:
        name = 'Unknown'
        color = (0, 0, 255)  # red for unrecognized face
    else:
        color = (0, 128, 0)  # dark green for recognized face

    # Draw a box around the face
    cv2.rectangle(frame, (left, top), (right, bottom), color, 2)

    # Draw a label with a name below the face
    cv2.rectangle(frame, (left, bottom - 35), (right, bottom), color, cv2.FILLED)
    cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 1.0, (255, 255, 255), 1)

# run_face_recognition_image(database)

# we can define accuracy  in different ways.

# 1. Recognition rate
# 2. false acceptance
# 3. true rejections

# Recognition rate is how many images correctly matching with the training images, while false acceptance is how many images (out side the dataset) are matching with the dataset images. true rejections are how many images (from the dataset) are not matching with the training dataset.

def cnn4(weights_path=None):
        model=Sequential()
        model.add(Conv2D(kernel_size=(3,3),filters=3,input_shape=(IMG_SIZE,IMG_SIZE,3),activation="relu"))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        model.add(Conv2D(kernel_size=(3,3),filters=3,activation="relu"))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        model.add(Conv2D(kernel_size=(3,3),filters=10))
#         model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        model.add(Conv2D(kernel_size=(3,3),filters=10))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        model.add(Flatten())
        model.add(Dropout(0.3))
        model.add(Dense(1000,activation="sigmoid"))
        model.add(Dense(100,activation="sigmoid"))
        model.add(Dense(1,activation="sigmoid"))
    #     model.summary()
        if weights_path:
            model.load_weights(weights_path)
        return model

model = cnn4('model_cnn4.h5')
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

def real_fake(img_array):
    new_img_array= cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
    x_test=new_img_array/255.0
    x_test = np.expand_dims(x_test, axis=0)
    probs= model.predict(x_test)
    labels = (probs>0.5).astype(np.int)
    return labels[0][0]

database = setup_database()
run_face_recognition_video(database)